In [1]:
from model import Model
import json
import os

/home/j1choi/Downloads/pyssem-main/pyssem/utils/simulation/scen_properties.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
# Load simulation configuration
with open('./example-purdue.json') as f:
  simulation_data = json.load(f)

In [3]:
scenario_props = simulation_data['scenario_properties']

In [4]:
scenario_props

{'start_date': '01/03/2024',
 'simulation_duration': 100,
 'steps': 100000,
 'min_altitude': 515,
 'max_altitude': 900,
 'v_shells': 2500000,
 'launch_function': 'Lambda',
 'integrator': 'BDF',
 'density_model': 'static_exp_dens_func',
 'LC': 0.1,
 'v_imp': 10.0,
 'fragment_spreading': False,
 'parallel_processing': False,
 'basline': False}

In [5]:
# Create an instance of the Model with the simulation parameters
model = Model(
    start_date=scenario_props["start_date"].split("T")[0],  # Assuming date is in ISO format
    simulation_duration=scenario_props["simulation_duration"],
    steps=scenario_props["steps"],
    min_altitude=scenario_props["min_altitude"],
    max_altitude=scenario_props["max_altitude"],
    v_shells=scenario_props["v_shells"],
    launch_function=scenario_props["launch_function"],
    integrator=scenario_props["integrator"],
    density_model=scenario_props["density_model"],
    LC=scenario_props["LC"],
    v_imp=scenario_props["v_imp"],
    fragment_spreading = scenario_props["fragment_spreading"]
)

SHELL COUNT:  3
R0_km [515.         515.74900413 516.49583905 517.2405204 ]


In [6]:
species = simulation_data["species"]
species_list = model.configure_species(species)

Splitting species N into 2 species with masses [0.64, 223].
Added 1 active species, 2 debris species, and 0 rocket body species to the simulation.
Pairing the following active species to debris classes for PMD modeling...
['S']
Matched species S to debris species N_223kg.
    Name: N_0.64kg
    pmd_linked_species: []
    Name: N_223kg
    pmd_linked_species: ['S']


Creating collision pairs: 100%|███████████████████| 6/6 [00:00<00:00, 45.47it/s]

Species pairing for  S N_0.64kg and sk  ['S', 'N_0.64kg', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'N_0.64kg'}
Species pairing for  S N_223kg and sk  ['S', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'N_223kg'}
Species pairing for  N_0.64kg N_223kg and sk  ['N_0.64kg', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'N_0.64kg'} {'N_223kg'}
Species pairing for  S S and sk  ['S', 'S', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'S'}
Species pairing for  N_0.64kg N_0.64kg and sk  ['N_0.64kg', 'N_0.64kg', 'N_0.64kg', 'N_223kg']
pairing {'N_0.64kg'} {'N_0.64kg'}
Species pairing for  N_223kg N_223kg and sk  ['N_223kg', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'N_223kg'} {'N_223kg'}


In [7]:
# Run the model
results = model.run_model()

baseline
PER YEAR 31557600.0 1000.0
PER YEAR 31557600.0 1000.0
PER YEAR 31557600.0 1000.0
Conversion of equations to lambda functions...
x0            S   N_0.64kg   N_223kg
0  15.657774  77.843113  5.672473
1  15.878601  77.907742  5.701144
2  16.100641  77.970844  5.729717
Integrating equations...
Model run completed successfully.


In [8]:
#model.create_plots()

In [9]:
# 전체 species 리스트 생성 (정확한 순서대로 구성)
all_species = species_list.species['active'] + species_list.species['debris']

initial_state = {}

# shell별, species별 초기 개체수 구성
for shell_idx in range(model.scenario_properties.n_shells):
    for species_idx, species_obj in enumerate(all_species):
        # 초기 개체수 정확히 설정 (정수값으로 명확히 변환)
        count = model.scenario_properties.x0.values[shell_idx, species_idx]
        initial_state[f'shell_{shell_idx+1}_{species_obj.sym_name}'] = count
        print(f'shell_{shell_idx+1}_{species_obj.sym_name}: {count}')

shell_1_S: 15.6577736678981
shell_1_N_0.64kg: 77.84311257696773
shell_1_N_223kg: 5.672472937222896
shell_2_S: 15.878601453161815
shell_2_N_0.64kg: 77.90774205306431
shell_2_N_223kg: 5.701144359293089
shell_3_S: 16.100641428714464
shell_3_N_0.64kg: 77.97084409201824
shell_3_N_223kg: 5.729717009629932


In [10]:
# collision 이벤트 정의 정확한 호출
from utils.simulation.discrete_event import define_collision_events_from_pairs, define_launch_events, define_pmd_events, define_drag_events
collision_events = define_collision_events_from_pairs(model, initial_state.copy())

# launch 이벤트 정의
launch_events = define_launch_events(all_species, model)

# PMD 이벤트 정의
pmd_events = define_pmd_events(all_species, model, initial_state.copy())

# drag 이벤트 정의
drag_events = define_drag_events(all_species, model, initial_state.copy())

colls
{'name': 'collision_S_N_0.64kg_lethal_shell_1', 'rate': 0.000376720407201032, 'jump': {'shell_1_S': -1, 'shell_1_N_0.64kg': 115.0}}
{'name': 'collision_S_N_0.64kg_disabling_shell_1', 'rate': 0.0188360203600516, 'jump': {'shell_1_S': -1, 'shell_1_N_223kg': 1}}
{'name': 'collision_S_N_0.64kg_lethal_shell_2', 'rate': 0.000383377985224091, 'jump': {'shell_2_S': -1, 'shell_2_N_0.64kg': 115.0}}
{'name': 'collision_S_N_0.64kg_disabling_shell_2', 'rate': 0.0191688992612045, 'jump': {'shell_2_S': -1, 'shell_2_N_223kg': 1}}
{'name': 'collision_S_N_0.64kg_lethal_shell_3', 'rate': 0.000390094571763904, 'jump': {'shell_3_S': -1, 'shell_3_N_0.64kg': 115.0}}
{'name': 'collision_S_N_0.64kg_disabling_shell_3', 'rate': 0.0195047285881952, 'jump': {'shell_3_S': -1, 'shell_3_N_223kg': 1}}
{'name': 'collision_S_N_223kg_lethal_shell_1', 'rate': 8.74119829430956e-5, 'jump': {'shell_1_S': -1, 'shell_1_N_223kg': -1, 'shell_1_N_0.64kg': 495.0}}
{'name': 'collision_S_N_223kg_disabling_shell_1', 'rate': 0.0

In [11]:
from utils.simulation.discrete_event import update_collision_event_rates, update_drag_events, update_pmd_events
import numpy as np

state = initial_state.copy()
time = 0
simulation_duration = model.scenario_properties.simulation_duration
event_log = []

while time < simulation_duration:
    # collision 이벤트 매번 업데이트
    collision_events = update_collision_event_rates(
        model.scenario_properties.collision_pairs,
        state,
        model.scenario_properties.n_shells
    )
    
    drag_events = update_drag_events(all_species, model, state)
    pmd_events = update_pmd_events(all_species, model, state)

    # 전체 이벤트 업데이트 (collision_events는 매번 업데이트, 다른 이벤트는 필요시 업데이트)
    all_events = collision_events + launch_events + pmd_events + drag_events

    total_rate = sum(event['rate'] for event in all_events)

    if total_rate == 0:
        print("No more events can occur.")
        break

    next_event_time = np.random.exponential(1 / total_rate)
    time += next_event_time

    probabilities = [event['rate'] / total_rate for event in all_events]
    selected_event = np.random.choice(all_events, p=probabilities)
    print(selected_event['name'])

    # 선택된 이벤트에 따라 상태 업데이트
    for species_shell, jump in selected_event['jump'].items():
        state[species_shell] += jump
        state[species_shell] = max(state[species_shell], 0)  # 음수 방지

    # 이벤트 로그 기록
    event_log.append({
        'time': time,
        'event': selected_event['name'],
        'state': state.copy()
    })


satellite_launch_shell_3
satellite_launch_shell_2
pmd_sat_success_shell_3
pmd_sat_success_shell_1
satellite_launch_shell_2
satellite_launch_shell_2
satellite_launch_shell_3
satellite_launch_shell_1
satellite_launch_shell_1
satellite_launch_shell_3
satellite_launch_shell_1
pmd_sat_success_shell_1
satellite_launch_shell_3
satellite_launch_shell_2
satellite_launch_shell_2
satellite_launch_shell_1
pmd_sat_success_shell_3
satellite_launch_shell_1
satellite_launch_shell_1
satellite_launch_shell_3
satellite_launch_shell_3
satellite_launch_shell_3
satellite_launch_shell_2
pmd_sat_success_shell_1
satellite_launch_shell_3
pmd_sat_success_shell_2
satellite_launch_shell_3
pmd_sat_success_shell_1
satellite_launch_shell_2
satellite_launch_shell_1
satellite_launch_shell_3
satellite_launch_shell_3
satellite_launch_shell_1
pmd_sat_success_shell_1
satellite_launch_shell_2
satellite_launch_shell_3
satellite_launch_shell_2
pmd_sat_success_shell_2
satellite_launch_shell_2
satellite_launch_shell_2
satellite

In [12]:
import numpy as np
import pandas as pd

# 기존 코드의 scen_times 사용
scen_times = model.scenario_properties.scen_times

# 이벤트 로그 DataFrame 변환
df_event_log = pd.DataFrame(event_log)

# 결과 배열 초기화
n_shells = model.scenario_properties.n_shells
n_species = len(all_species)
result_array = np.zeros((n_species * n_shells, len(scen_times)))

# species_shell 이름 구성
species_shell_names = [f'shell_{k+1}_{sp.sym_name}' for sp in all_species for k in range(n_shells)]

# 이벤트 인덱스 초기화
current_idx = 0
for step_idx, t in enumerate(scen_times):
    # 다음 이벤트가 현재 시나리오 시간(t)을 넘기지 않을 때까지 이벤트 인덱스 업데이트
    while current_idx + 1 < len(df_event_log) and df_event_log['time'].iloc[current_idx + 1] <= t:
        current_idx += 1
    
    # 현재 상태를 각 species_shell에 저장
    current_state = df_event_log['state'].iloc[current_idx]
    for idx, species_shell in enumerate(species_shell_names):
        result_array[idx, step_idx] = current_state.get(species_shell, 0)

# 결과 배열 형태 확인
print(result_array.shape)  # (n_species * n_shells, n_steps)



(9, 100000)


In [13]:
results.output.y

array([[15.65777367, 15.66487711, 15.67198056, ..., 46.73361559,
        46.73346322, 46.73331085],
       [15.87860145, 15.88563215, 15.89266285, ..., 46.58512708,
        46.58497431, 46.58482154],
       [16.10064143, 16.10757104, 16.11450066, ..., 46.3341948 ,
        46.33404334, 46.33389187],
       ...,
       [ 5.67247294,  5.67265231,  5.67283169, ..., 83.26264078,
        83.2639716 , 83.26530245],
       [ 5.70114436,  5.70132636,  5.70150836, ..., 83.19886796,
        83.20019835, 83.20152877],
       [ 5.72971701,  5.72990165,  5.73008629, ..., 82.82088646,
        82.82220818, 82.82352992]], shape=(9, 100000))

In [14]:
result_array

array([[15.65777367, 15.65777367, 15.65777367, ..., 36.65777367,
        36.65777367, 36.65777367],
       [15.87860145, 15.87860145, 15.87860145, ..., 37.87860145,
        37.87860145, 37.87860145],
       [17.10064143, 17.10064143, 17.10064143, ..., 48.10064143,
        48.10064143, 48.10064143],
       ...,
       [ 5.67247294,  5.67247294,  5.67247294, ..., 91.67247294,
        91.67247294, 91.67247294],
       [ 5.70114436,  5.70114436,  5.70114436, ..., 67.70114436,
        67.70114436, 67.70114436],
       [ 5.72971701,  5.72971701,  5.72971701, ..., 62.72971701,
        62.72971701, 62.72971701]], shape=(9, 100000))

In [15]:
results.output.y = result_array

In [16]:
import matplotlib.pyplot as plt
import numpy as np
scenario_properties = model.scenario_properties
output = results.output
#results.output.t = np.interp(np.linspace(0, 1600, 1600*10).tolist(), range(1600), results.output.t)
os.makedirs('figures', exist_ok=True)


# Plot each species across all shells
n_species = 3
num_shells = scenario_properties.n_shells

fig, axes = plt.subplots(1, 3, figsize=(20, 12))

# Colormap setup
cmap = plt.cm.viridis  # Choose your colormap
norm = plt.Normalize(vmin=0, vmax=num_shells - 1)
colors = [cmap(norm(sh)) for sh in range(num_shells)]

for species_index in range(n_species):
    ax = axes.flatten()[species_index]
    species_data = output.y[species_index * num_shells:(species_index + 1) * num_shells]

    for shell_index in range(num_shells):
        color = colors[shell_index]
        ax.plot(output.t, species_data[shell_index], color = color, label=f'Shell {shell_index + 1}')

    total = np.sum(species_data, axis=0)
    ax.set_title(f'{scenario_properties.species_names[species_index]}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')

#ax.set_ylim([0, 1e8])
plt.suptitle('Species All Shells')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('figures/species_all_shells.png')
plt.close(fig)

# Plot total objects over time for each species and total
species_names = scenario_properties.species_names
plt.figure(figsize=(10, 6))

total_objects_all_species = np.zeros_like(output.t)

for i in range(n_species):
    start_idx = i * num_shells
    end_idx = start_idx + num_shells
    total_objects_per_species = np.sum(output.y[start_idx:end_idx, :], axis=0)
    if i == 0:
        print(total_objects_per_species)
    plt.plot(output.t, total_objects_per_species, label=f'{species_names[i]}')
    total_objects_all_species += total_objects_per_species

plt.plot(output.t, total_objects_all_species, label='Total All Species', color='k', linewidth=2, linestyle='--')
#plt.ylim(0, 1e8)
plt.xlabel('Time')
plt.ylabel('Total Number of Objects')
plt.title('Objects Over Time for Each Species and Total')
plt.xlim(0,100)
#plt.ylim(0, 100)
plt.legend()
plt.tight_layout()
plt.savefig('figures/total_objects_over_time.png')
plt.close()

[ 48.63701655  48.63701655  48.63701655 ... 122.63701655 122.63701655
 122.63701655]
